In [3]:
import sklearn
sklearn.__version__

'0.24.2'

In [4]:
!pip install -U scikit-learn>=1.0.2

In [1]:
import sklearn
sklearn.__version__

'1.2.0'

In [2]:
!pip install imblearn

In [3]:
import imblearn
print(imblearn.__version__)

0.10.0


In [4]:
import numpy as np 
import pandas as pd 

In [14]:
df=pd.read_csv("C:/Users/10697598/Desktop/Others/NewDataSets/predictive_maintenance.csv")

In [15]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [16]:
df.drop(df.columns[:2],axis=1,inplace=True)

In [17]:
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,M,298.1,308.6,1551,42.8,0,0,No Failure
1,L,298.2,308.7,1408,46.3,3,0,No Failure
2,L,298.1,308.5,1498,49.4,5,0,No Failure
3,L,298.2,308.6,1433,39.5,7,0,No Failure
4,L,298.2,308.7,1408,40.0,9,0,No Failure


In [10]:
from sklearn .preprocessing import LabelEncoder
encode=LabelEncoder()

In [18]:
df.loc[:,df.columns[-1]]=encode.fit_transform(df.loc[:,df.columns[-1]])

In [19]:
encode.classes_

array(['Heat Dissipation Failure', 'No Failure', 'Overstrain Failure',
       'Power Failure', 'Random Failures', 'Tool Wear Failure'],
      dtype=object)

In [20]:
df.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,M,298.1,308.6,1551,42.8,0,0,1
1,L,298.2,308.7,1408,46.3,3,0,1
2,L,298.1,308.5,1498,49.4,5,0,1
3,L,298.2,308.6,1433,39.5,7,0,1
4,L,298.2,308.7,1408,40.0,9,0,1


In [21]:
from sklearn.model_selection import train_test_split
X_train , X_test = train_test_split(df,test_size = 0.3)

In [22]:
df = X_train

Need to min max Scaler on Roatation speed and Standard scaller on all
### Use MinMaxScaler as your default
### Use RobustScaler if you have outliers and can handle a larger range
### Use StandardScaler if you need normalized features
### Use Normalizer sparingly - it normalizes rows, not columns

In [23]:
df.drop(df.columns[-2],axis=1,inplace=True)

In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
df=pd.get_dummies(df,drop_first=True)

In [26]:
X_test = pd.get_dummies(X_test,drop_first=True)

In [27]:
X_test.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,Type_L,Type_M
6090,300.9,310.8,1708,29.1,14,0,1,1,0
4891,303.7,312.3,1756,26.4,190,0,1,0,1
4645,303.3,311.4,1732,25.6,232,0,1,1,0
329,297.7,308.5,1720,29.9,210,0,1,0,0
6926,301.1,311.6,1417,53.4,0,0,1,0,0


In [28]:
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler

In [29]:
df=df.sample(frac=1).reset_index(drop=True)

In [30]:
df.columns=df.columns.astype("str")

In [31]:
col_name=df.columns.to_list()

In [32]:
target_col=col_name.pop(-3)

In [33]:
feature_col=col_name

In [34]:
target_col

'Failure Type'

In [35]:
feature_col

['Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Type_L',
 'Type_M']

In [36]:
y=df["Failure Type"].values

In [37]:
from imblearn.combine import SMOTETomek
from collections import Counter

In [38]:
df_train=df.drop(df.columns[-3],axis=1)

In [39]:
df_train_val=df_train.values

In [40]:
smk=SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(df_train_val,y)

In [41]:
X_res.shape,y_res.shape

((40546, 7), (40546,))

In [42]:
print(f"Orignal Dataset Shape {Counter(y)}")
print(f"Resampled Dataset Shape {Counter(y_res)}")

Orignal Dataset Shape Counter({1: 6766, 0: 80, 3: 62, 2: 52, 5: 28, 4: 12})
Resampled Dataset Shape Counter({2: 6766, 3: 6763, 0: 6762, 4: 6759, 5: 6755, 1: 6741})


In [43]:
df_new=pd.concat([pd.DataFrame(X_res),pd.DataFrame(y_res)],axis=1)

In [45]:
col_new=df_train.columns
col_new=col_new.to_list()
col_new.append(target_col)
df_new.columns=col_new
df_new.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_L,Type_M,Failure Type
0,295.5,305.9,1542.0,36.2,12.0,1.0,0.0,1
1,304.0,313.2,1416.0,46.0,128.0,1.0,0.0,1
2,300.5,311.8,1591.0,42.0,182.0,0.0,1.0,1
3,302.5,312.0,1487.0,46.1,175.0,0.0,1.0,1
4,298.0,308.1,1403.0,44.8,148.0,0.0,1.0,1


In [47]:
df_new.shape

(40546, 8)

In [48]:
X_test.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,Type_L,Type_M
6090,300.9,310.8,1708,29.1,14,0,1,1,0
4891,303.7,312.3,1756,26.4,190,0,1,0,1
4645,303.3,311.4,1732,25.6,232,0,1,1,0
329,297.7,308.5,1720,29.9,210,0,1,0,0
6926,301.1,311.6,1417,53.4,0,0,1,0,0


In [50]:
col_new

['Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Type_L',
 'Type_M',
 'Failure Type']

In [56]:
df_new['Split'] = "train"

In [54]:
df_new2 = X_test[col_new] 

In [57]:
df_new2['Split'] = "test"

C:\Users\10697598\AppData\Local\Temp/ipykernel_3140/2660398586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new2['Split'] = "test"


In [59]:
df_new2.shape

(3000, 9)

In [60]:
df_new.shape

(40546, 9)

In [66]:
df_new1 = df_new
df_final = df_new.append(df_new2, ignore_index=True)

In [67]:
df_final.shape

(43546, 9)

In [68]:
df_final.to_csv('C:/Users/10697598/Desktop/Others/NewDataSets/predictive_maintenance_upsample.csv')